<a href="https://colab.research.google.com/github/Kavyabikkasani/AI_Internship_oct24/blob/main/Gradio_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

def load_model(model_name="google/flan-t5-small"):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        return tokenizer, model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Global model loading
tokenizer, model = load_model()

def extract_comprehensive_details(review_text):
    if not tokenizer or not model:
        return "Model not loaded correctly."

    prompt = f"""
    Perform an exhaustive analysis of this review. Extract EVERY possible detail:

    Review Text: {review_text}

    MANDATORY EXTRACTION REQUIREMENTS:
    A. COMPLETE CUSTOMER INFORMATION
    - Full Name (if mentioned)
    - Customer Identifier
    - Contact Details (if available)
    - Age/Demographics (if implied)

    B. PRODUCT SPECIFICS
    - Exact Product Name
    - Product Model Number
    - Manufacturer/Brand
    - Product Category
    - Specific Product Features

    C. DETAILED SENTIMENT ANALYSIS
    - Overall Emotional Tone
    - Sentiment Intensity (0-100 scale)
    - Emotional Triggers in Review

    D. COMPREHENSIVE PRODUCT EVALUATION
    1. POSITIVE ASPECTS
    - List EVERY mentioned positive feature
    - Specific praise points
    - Standout characteristics
    - Direct positive quotes

    2. NEGATIVE ASPECTS
    - List EVERY mentioned negative feature
    - Specific criticism points
    - Areas of disappointment
    - Direct negative quotes

    3. DETAILED FEATURE BREAKDOWN
    - Performance Analysis
    - Quality Assessment
    - Value for Money
    - Comparative Remarks

    E. CONTEXTUAL INSIGHTS
    - Implied Use Cases
    - Purchase Context
    - Customer Expectations
    - Potential Recommendations

    Provide maximum possible detail for EACH section.
    """

    try:
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding=True
        )

        outputs = model.generate(
            **inputs,
            max_length=1024,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    except Exception as e:
        return f"Error processing review: {str(e)}"

def gradio_review_extraction(file):
    if not file:
        return "No file uploaded."

    try:
        # Read file content
        if hasattr(file, 'name'):
            with open(file.name, 'r', encoding='utf-8') as f:
                review_text = f.read().strip()
        elif hasattr(file, 'read'):
                review_text = file.read().decode('utf-8').strip()
        else:
            review_text = str(file).strip()

        return extract_comprehensive_details(review_text)

    except Exception as e:
        return f"Error processing uploaded file: {str(e)}"

def create_gradio_interface():
    interface = gr.Interface(
        fn=gradio_review_extraction,
        inputs=gr.File(label="Upload Customer Review File"),
        outputs=gr.Textbox(label="Comprehensive Review Analysis"),
        title="Exhaustive Review Details Extractor",
        description="Upload a review file for maximum detail extraction.",
        theme='default'
    )
    return interface

# Main execution
if __name__ == "__main__":
    interface = create_gradio_interface()
    interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6cb3892abef521a4bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
